In [1]:
import pandas as pd
import ast
import boto3

In [2]:
poker = pd.read_csv("poker_hand_dataset.csv", low_memory=False)
poker.head(10)

,S1,C1,S2,C2,S3,C3,S4,C4,S5,C5,CLASS
0,1,1,1,13,2,4,2,3,1,12,0
1,3,12,3,2,3,11,4,5,2,5,1
2,1,9,4,6,1,4,3,2,3,9,1
3,1,4,3,13,2,13,2,1,3,6,1
4,3,10,2,7,1,2,2,11,4,9,0
5,1,3,4,5,3,4,1,12,4,6,0
6,2,6,4,11,2,3,4,9,1,7,0
7,3,2,4,9,3,7,4,3,4,5,0
8,4,4,3,13,1,8,3,9,3,10,0
9,1,9,3,8,4,4,1,7,3,5,0


In [3]:
print("Total rows in Poker Hand dataset is:", len(poker))

Total rows in Poker Hand dataset is: 1000000


In [6]:
# AWS configuration (no need to specify credentials)
AWS_REGION = 'us-east-1'
S3_BUCKET_NAME = 'poker-hand-project'
S3_KEY = 'test/'
filename = 'poker.csv'

In [7]:
poker_csv = poker.to_csv(header=True, index=False)
s3 = boto3.client('s3', region_name=AWS_REGION)
s3.put_object(Body=poker_csv, Bucket=S3_BUCKET_NAME, Key=f'{S3_KEY}{filename}')

{'ResponseMetadata': {'RequestId': '5RSTSE9JG3JD4JQN',
  'HostId': 'pdDmdujW7JPLTrMzC7c+ERcpuDdzms0MdSAubIc8RGRGJEhGVoimmlqUF1XJsEgW2n5fgpzjozAqicTW8mZHfA==',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'pdDmdujW7JPLTrMzC7c+ERcpuDdzms0MdSAubIc8RGRGJEhGVoimmlqUF1XJsEgW2n5fgpzjozAqicTW8mZHfA==',
   'x-amz-request-id': '5RSTSE9JG3JD4JQN',
   'date': 'Wed, 19 Mar 2025 03:33:39 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"df080bad7b4eafe69fde8315068afd74"',
   'x-amz-checksum-crc32': 'EbuBlA==',
   'x-amz-checksum-type': 'FULL_OBJECT',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"df080bad7b4eafe69fde8315068afd74"',
 'ChecksumCRC32': 'EbuBlA==',
 'ChecksumType': 'FULL_OBJECT',
 'ServerSideEncryption': 'AES256'}

In [8]:
# Athena configuration
DATABASE_NAME = 'default'
TABLE_NAME = 'poker-hand-table'

# Initialize Athena client (no need to specify credentials)
athena_client = boto3.client('athena', region_name=AWS_REGION)

In [26]:
# Define the table creation query
create_table_query = f"""
CREATE EXTERNAL TABLE IF NOT EXISTS `{DATABASE_NAME}`.`{TABLE_NAME}` (
    S1 INT,
    C1 INT,
    S2 INT,
    C2 INT,
    S3 INT,
    C3 INT,
    S4 INT,
    C4 INT,
    S5 INT,
    C5 INT,
    CLASS INT
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ',' 
STORED AS TEXTFILE
LOCATION 's3://{S3_BUCKET_NAME}/{S3_KEY}/';
"""

# Print the query
print(create_table_query.strip())

CREATE EXTERNAL TABLE IF NOT EXISTS `default`.`poker-hand-table` (
    S1 INT,
    C1 INT,
    S2 INT,
    C2 INT,
    S3 INT,
    C3 INT,
    S4 INT,
    C4 INT,
    S5 INT,
    C5 INT,
    CLASS INT
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ',' 
STORED AS TEXTFILE
LOCATION 's3://poker-hand-project/test//';


In [27]:
#Executing create table query
response = athena_client.start_query_execution(
    QueryString=create_table_query.strip(),
    QueryExecutionContext={'Database': DATABASE_NAME},
    ResultConfiguration=
    {'OutputLocation': f's3://{S3_BUCKET_NAME}/query_results/'}
)

In [28]:
import time

# Function to run Athena query
def run_athena_query(query, database, output_location):
    response = athena_client.start_query_execution(
        QueryString=query,
        QueryExecutionContext={
            'Database': database
        },
        ResultConfiguration={
            'OutputLocation': output_location
        }
    )
    return response['QueryExecutionId']

# Function to check query status
def check_query_status(query_execution_id):
    while True:
        response = athena_client.get_query_execution(QueryExecutionId=query_execution_id)
        status = response['QueryExecution']['Status']['State']
        if status == 'SUCCEEDED':
            print("Query succeeded!")
            break
        elif status in ['FAILED', 'CANCELLED']:
            print(f"Query {status.lower()}.")
            raise Exception(f"Query failed or was cancelled: {response}")
        time.sleep(2)

# Function to get query results and convert to pandas DataFrame
def get_query_results_as_dataframe(query_execution_id):
    # Fetching the query results
    results_paginator = athena_client.get_paginator('get_query_results')
    results_iter = results_paginator.paginate(QueryExecutionId=query_execution_id)

    # Initialize a list to store rows and columns for DataFrame
    columns = []
    rows = []

    # Process the result pages
    for results_page in results_iter:
        # Get column info from the first page
        if not columns:
            columns = [col['Label'] for col in results_page['ResultSet']['ResultSetMetadata']['ColumnInfo']]

        # Skip the first row of the first page (column headers)
        for row in results_page['ResultSet']['Rows'][1:]:
            rows.append([col.get('VarCharValue', None) for col in row['Data']])

    # Create DataFrame from the results
    df = pd.DataFrame(rows, columns=columns)
    return df


In [29]:
# Query details
query = f'SELECT * FROM "default"."{TABLE_NAME}" limit 10;'
database = 'default'
output_location = f's3://{S3_BUCKET_NAME}/query_results/'  # Change to your S3 bucket

# Run the query
query_execution_id = run_athena_query(query, database, output_location)

# Check the status of the query
check_query_status(query_execution_id)

# Fetch the results as a pandas DataFrame
query_results = get_query_results_as_dataframe(query_execution_id)

Query succeeded!


In [30]:
query_results

,s1,c1,s2,c2,s3,c3,s4,c4,s5,c5,class
0,None,None,None,None,None,None,None,None,None,None,None
1,1,1,1,13,2,4,2,3,1,12,0
2,3,12,3,2,3,11,4,5,2,5,1
3,1,9,4,6,1,4,3,2,3,9,1
4,1,4,3,13,2,13,2,1,3,6,1
5,3,10,2,7,1,2,2,11,4,9,0
6,1,3,4,5,3,4,1,12,4,6,0
7,2,6,4,11,2,3,4,9,1,7,0
8,3,2,4,9,3,7,4,3,4,5,0
9,4,4,3,13,1,8,3,9,3,10,0
